## Deliverable 2. Create a Customer Travel Destinations Map.

In [4]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import os
import numpy as np

# Import API key
import sys
sys.path.append("..")
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [5]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv(os.path.join("..", "Weather_Database", "WeatherPy_database.csv"))
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Punta Arenas,CL,-53.1500,-70.9167,51.82,87,75,10.36,light rain
1,1,Bengkulu,ID,-3.8004,102.2655,75.90,90,99,4.27,overcast clouds
2,2,Ourossogui,SN,15.6064,-13.3211,85.55,10,93,10.83,overcast clouds
3,3,Doume,CM,4.2333,13.4500,73.33,33,14,1.14,few clouds
4,4,Sambava,MG,-14.2667,50.1667,77.79,80,100,4.38,overcast clouds


In [6]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimun temperature you would like for your trip? "))
max_temp = float(input("What is the maximun temperature you would like for your trip? "))

What is the minimun temperature you would like for your trip? 75
What is the maximun temperature you would like for your trip? 90


In [7]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
cities_preferred_df = city_data_df.loc[(city_data_df["Max Temp"] >= min_temp) & (city_data_df["Max Temp"] <= max_temp)]
cities_preferred_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Bengkulu,ID,-3.8004,102.2655,75.90,90,99,4.27,overcast clouds
2,2,Ourossogui,SN,15.6064,-13.3211,85.55,10,93,10.83,overcast clouds
4,4,Sambava,MG,-14.2667,50.1667,77.79,80,100,4.38,overcast clouds
9,9,Rikitea,PF,-23.1203,-134.9692,77.94,74,100,5.93,overcast clouds
14,14,Atuona,PF,-23.1203,-134.9692,77.94,74,100,5.93,overcast clouds


In [8]:
# 4a. Determine if there are any empty rows.
print(len(cities_preferred_df))
cities_preferred_df.count()

231


City_ID                231
City                   231
Country                231
Lat                    231
Lng                    231
Max Temp               231
Humidity               231
Cloudiness             231
Wind Speed             231
Current Description    231
dtype: int64

In [9]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = cities_preferred_df.dropna(axis="index", how="any")
print(len(clean_df))
clean_df.head()

231


,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Bengkulu,ID,-3.8004,102.2655,75.90,90,99,4.27,overcast clouds
2,2,Ourossogui,SN,15.6064,-13.3211,85.55,10,93,10.83,overcast clouds
4,4,Sambava,MG,-14.2667,50.1667,77.79,80,100,4.38,overcast clouds
9,9,Rikitea,PF,-23.1203,-134.9692,77.94,74,100,5.93,overcast clouds
14,14,Atuona,PF,-23.1203,-134.9692,77.94,74,100,5.93,overcast clouds


In [10]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Bengkulu,ID,75.90,overcast clouds,-3.8004,102.2655,
2,Ourossogui,SN,85.55,overcast clouds,15.6064,-13.3211,
4,Sambava,MG,77.79,overcast clouds,-14.2667,50.1667,
9,Rikitea,PF,77.94,overcast clouds,-23.1203,-134.9692,
14,Atuona,PF,77.94,overcast clouds,-23.1203,-134.9692,
24,Paita,PE,84.76,clear sky,-5.0892,-81.1144,
25,Conceicao Do Araguaia,BR,85.30,moderate rain,-8.2578,-49.2647,
31,Banda Aceh,ID,75.90,overcast clouds,-3.8004,102.2655,
33,Maceio,BR,85.30,moderate rain,-8.2578,-49.2647,
36,Mataura,PF,77.94,overcast clouds,-23.1203,-134.9692,


In [11]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found...skipping the city.")
        hotel_df.loc[index, "Hotel Name"] = np.nan
        

Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.


In [12]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna(axis="index", how="any")
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Bengkulu,ID,75.90,overcast clouds,-3.8004,102.2655,Grage Hotel Bengkulu
2,Ourossogui,SN,85.55,overcast clouds,15.6064,-13.3211,Hotel welma
4,Sambava,MG,77.79,overcast clouds,-14.2667,50.1667,Melrose
9,Rikitea,PF,77.94,overcast clouds,-23.1203,-134.9692,People ThankYou
14,Atuona,PF,77.94,overcast clouds,-23.1203,-134.9692,People ThankYou
24,Paita,PE,84.76,clear sky,-5.0892,-81.1144,Hotel Náutico de Paita
25,Conceicao Do Araguaia,BR,85.30,moderate rain,-8.2578,-49.2647,Tarumã Tropical Hotel
31,Banda Aceh,ID,75.90,overcast clouds,-3.8004,102.2655,Grage Hotel Bengkulu
33,Maceio,BR,85.30,moderate rain,-8.2578,-49.2647,Tarumã Tropical Hotel
36,Mataura,PF,77.94,overcast clouds,-23.1203,-134.9692,People ThankYou


In [13]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [14]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [15]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))